# Experiment 2: No-fire December set + HRRR

This notebook focuses on running Experiment 2, which uses data from December (no-fire period) combining PWWB, AirNow, and HRRR datasets.

# Data parameters

In [1]:
# define bounding box
lat_bottom, lat_top = 33.5, 34.5
lon_bottom, lon_top = -118.75, -117.0
extent = (lon_bottom, lon_top, lat_bottom, lat_top)

# input data shape
dim = 200
frames_per_sample = 5

# date range of data - two weeks of December for this experiment 2024-12-08-00", "2024-12-21-00"
dec_start_date, dec_end_date = "2024-12-01-00", "2025-01-01-00"

# Data ingestion and preprocessing

In [2]:
# python nonsense that allows you to import from sibling directories
import sys
sys.path.append("..")

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dotenv import load_dotenv

# Import the new PWWB implementation and dataset manager
from libs.pwwb import PWWBData
from libs.pwwb.utils.dataset_manager import create_dataset_manager

# Import the AirNow data class
from libs.airnowdata import AirNowData

# Import the HRRR data class
from libs.hrrrdata import HRRRData

# Load environment variables (API keys, credentials)
load_dotenv()

# split data
def train_test_split(X, train_size=0.75):
    split_idx = int(X.shape[0] * train_size)
    X_train, X_test = X[:split_idx], X[split_idx:]
    
    return X_train, X_test

# scale training data, then scale test data based on training data stats
from sklearn.preprocessing import StandardScaler
def std_scale(X_train, X_test):
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(X_train.reshape(-1, 1)).reshape(X_train.shape)
    scaled_test = scaler.transform(X_test.reshape(-1, 1)).reshape(X_test.shape)

    return scaled_train, scaled_test

/home/moh/miniconda3/envs/hysplitrevised/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create output directory for results
output_dir = "experiment_output"
os.makedirs(output_dir, exist_ok=True)

# Create dataset manager
manager = create_dataset_manager(
    registry_file="experiment2_registry.json",
    cache_dir="data/pwwb_cache/"
)

# List existing datasets
print("Existing datasets:")
try:
    display(manager.list_datasets())
except:
    print("No existing datasets found.")

Existing datasets:


,name,created,description,start_date,end_date,channels
0,dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY,2025-05-20T22:58:50.691219,"December 2024 - two weeks with MAIAC, TROPOMI ...",2024-12-01-00,2025-01-01-00,"maiac, tropomi, metar"


In [7]:
# Adjust end date for AirNow
dec_end_date_adj = pd.to_datetime(dec_end_date) - pd.Timedelta(hours=1)

# Dataset name and description
dataset_name = "dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY"
dataset_desc = "December 2024 - two weeks with MAIAC, TROPOMI NO2, METAR Wind U/V components only"

# ========== 1. Load December PWWB Data ==========
print("\nLoading December PWWB data...")

# Check if dataset already exists in the registry
if manager.get_dataset_info(dataset_name) is not None:
    print(f"Dataset '{dataset_name}' already exists, loading from cache...")
    dec_pwwb = manager.load_dataset(dataset_name, PWWBData)
else:
    print(f"Dataset '{dataset_name}' not found, creating new one...")
    # Create the dataset with the specified channels
    dec_pwwb = manager.create_dataset(
        name=dataset_name,
        description=dataset_desc,
        PWWBData_class=PWWBData,
        start_date=dec_start_date,
        end_date=dec_end_date,
        extent=extent,
        frames_per_sample=frames_per_sample,
        dim=dim,
        include_channels={
            'maiac': True,                     # Include MAIAC AOD
            'tropomi': ['TROPOMI_NO2'],        # Only include NO2 from TROPOMI
            'metar': ['METAR_Wind_U', 'METAR_Wind_V'],  # Only wind components from METAR
            'modis_fire': False,               # Exclude MODIS fire data
            'merra2': False                    # Exclude MERRA2 data
        },
        verbose=True,
        output_dir=output_dir
    )
    # Save the dataset
    dec_pwwb.save_data()

# Get the data and channel info
X_dec_pwwb = dec_pwwb.data
channel_info = dec_pwwb.get_channel_info()
print(f"✓ December PWWB data shape: {X_dec_pwwb.shape}")
print(f"  Channels: {channel_info['channel_names']}")

# ========== 2. Load December AirNow Data ==========
print("\nLoading December AirNow data...")
dec_airnow = AirNowData(
    start_date=dec_start_date,
    end_date=dec_end_date_adj,
    extent=extent,
    airnow_api_key=os.getenv('AIRNOW_API_KEY'),
    frames_per_sample=frames_per_sample,
    dim=dim,
    elevation_path="../libs/inputs/elevation.npy",
    mask_path="../libs/inputs/mask.npy",
    force_reprocess=False
)
X_dec_airnow = dec_airnow.data
Y_dec = dec_airnow.target_stations
print(f"✓ December AirNow data shape: {X_dec_airnow.shape}")
if Y_dec is not None:
    print(f"  December target stations shape: {Y_dec.shape}")
else:
    print("  No December target stations available")

# ========== 3. Load December HRRR Data ==========
print("\nLoading December HRRR data...")
dec_hrrr = HRRRData(
    start_date=dec_start_date,
    end_date=dec_end_date_adj,
    extent=extent,
    extent_name='la_region',
    product='COLMD',
    frames_per_sample=frames_per_sample,
    dim=dim,
    verbose=True,
    sample_setting=2
)
# Convert units from kg to micrograms (1e9)
X_dec_hrrr = dec_hrrr.data
print(f"✓ December HRRR data shape: {X_dec_hrrr.shape}")


Loading December PWWB data...
Dataset 'dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY' already exists, loading from cache...
Using cache prefix: dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY_
Initialized PWWBData with 744 hourly timestamps
Date range: 2024-12-01 00:00:00 to 2024-12-31 23:00:00
Channels included: ['maiac', 'tropomi', 'metar']
TROPOMI channels: ['TROPOMI_NO2']
METAR channels: ['METAR_Wind_U', 'METAR_Wind_V']
Processing MAIAC AOD data...
Loading cached MAIAC AOD data from data/pwwb_cache/dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY_maiac_aod_data.npy


/home/moh/wildfire/hrrr-smoke-viz/pwwb-experiments/../libs/pwwb/pwwb_data.py:115: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  self.timestamps = pd.date_range(self.start_date, self.end_date, freq='H')


Processing TROPOMI data...
Including TROPOMI channels: ['TROPOMI_NO2']
Loading cached TROPOMI data from data/pwwb_cache/dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY_tropomi_no2_data.npy
Processing METAR meteorological data...
Initialized MetarDataSource with 2 channels: ['METAR_Wind_U', 'METAR_Wind_V']
Will fetch these raw variables: ['sped', 'drct']
Will calculate wind U/V components from speed/direction
Loading cached data from data/pwwb_cache/dec2024_MAIC_TROPOMI_N02_METAR_WIND_UV_ONLY_metar_u_v_data.npy
Final data shape: (740, 5, 200, 200, 4)

Channel Statistics:

Channel 0: MAIAC_AOD
  Min: -28672.000000000007
  Max: -11408.21815610717
  Mean: -22674.333462043887
  Std: 5129.642805682386
  Data coverage: 100.00% (40000/40000 non-zero pixels)

Channel 1: TROPOMI_NO2
  Min: 8.291821094530346e-06
  Max: 0.0005449624637115875
  Mean: 0.00012861194879700214
  Std: 8.197002367478071e-05
  Data coverage: 100.00% (40000/40000 non-zero pixels)

Channel 2: METAR_Wind_U
  Min: 2.8166876380389

CalledProcessError: Command '/home/moh/miniconda3/envs/hysplitrevised/bin/wgrib2 -s /home/moh/data/hrrr/20241224/subset_70e4105b__hrrr.t07z.wrfsfcf01.grib2' returned non-zero exit status 8.

In [ ]:
# ========== 4. Create Experiment 2 dataset ==========
print("\nCreating Experiment 2 dataset...")

# Experiment 2: No-fire December set + HRRR
print("  Experiment 2: No-fire December set + HRRR")
X_exp2 = np.concatenate([X_dec_pwwb, X_dec_airnow, X_dec_hrrr], axis=-1)
print(f"    Combined shape: {X_exp2.shape}")

# Display the number of channels from each source
pwwb_channels = X_dec_pwwb.shape[4]
airnow_channels = X_dec_airnow.shape[4]
hrrr_channels = X_dec_hrrr.shape[4]
print(f"    PWWB channels: {pwwb_channels}, AirNow channels: {airnow_channels}, HRRR channels: {hrrr_channels}, Total: {X_exp2.shape[4]}")

In [ ]:
# ========== 5. Train/Test Split for experiment ==========
print("\nCreating train/test splits for experiment...")
# Experiment 2 splits
X_exp2_train, X_exp2_test = train_test_split(X_exp2, train_size=0.75)
Y_dec_train, Y_dec_test = train_test_split(Y_dec, train_size=0.75)
print(f"  Experiment 2: Train={X_exp2_train.shape}, Test={X_exp2_test.shape}")

# ========== 6. Standardize data ==========
print("\nStandardizing data...")

# Experiment 2 standardization
X_exp2_train_scaled, X_exp2_test_scaled = std_scale(X_exp2_train, X_exp2_test)
print(f"  Experiment 2: Scaled train={X_exp2_train_scaled.shape}, test={X_exp2_test_scaled.shape}")

In [ ]:
# ========== 7. Save prepared datasets ==========
print("\nSaving prepared dataset...")

# Create directory for experiment
exp_dir = os.path.join(output_dir, "experiment2")
os.makedirs(exp_dir, exist_ok=True)

# Save Experiment 2 data
np.save(os.path.join(exp_dir, "X_train.npy"), X_exp2_train_scaled)
np.save(os.path.join(exp_dir, "X_test.npy"), X_exp2_test_scaled)
np.save(os.path.join(exp_dir, "y_train.npy"), Y_dec_train)
np.save(os.path.join(exp_dir, "y_test.npy"), Y_dec_test)

# Save metadata
metadata = {
    "date_range": f"{dec_start_date} to {dec_end_date}",
    "extent": extent,
    "dim": dim,
    "frames_per_sample": frames_per_sample,
    "pwwb_channels": channel_info['channel_names'],
    "airnow_channels": ["AirNow_PM25"],
    "hrrr_channels": ["HRRR_COLMD"]
}

with open(os.path.join(exp_dir, "metadata.json"), "w") as f:
    import json
    json.dump(metadata, f, indent=2)

print("\n✓ Dataset prepared and saved!")

# Data visualization

In [ ]:
# Function to visualize data from experiment
def visualize_experiment_data(X, y, channel_names=None, sample_idx=None):
    """Visualize data from the experiment"""
    # Get a random sample if none provided
    if sample_idx is None:
        np.random.seed(42)
        sample_idx = np.random.choice(range(len(X)), size=1)[0]
    
    # Get channel information
    n_channels = X.shape[4]
    n_frames = X.shape[1]
    
    # Use provided channel names or create default ones
    if channel_names is None or len(channel_names) != n_channels:
        channel_names = [f"Channel {i}" for i in range(n_channels)]
    
    # Create figure
    fig, axes = plt.subplots(n_channels, n_frames, figsize=(3*n_frames, 2*n_channels))
    if n_channels == 1:
        axes = axes.reshape(1, -1)
    
    # Plot each channel and frame
    for c in range(n_channels):
        for f in range(n_frames):
            ax = axes[c, f]
            im = ax.imshow(X[sample_idx, f, :, :, c])
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            if f == 0:
                ax.set_ylabel(channel_names[c])
            ax.set_title(f"Frame {f+1}")
    
    # Set title
    plt.suptitle(f"Experiment 2: No-fire December set + HRRR\nSample {sample_idx}")
    plt.tight_layout()
    plt.show()
    
    # Print target values
    if y is not None:
        print(f"Target values: {y[sample_idx]}")

# Create combined channel names list
all_channel_names = channel_info['channel_names'] + ["AirNow_PM25"] + ["HRRR_COLMD"]

# Visualize a sample from the experiment
print("Visualizing data...")
visualize_experiment_data(X_exp2_train_scaled, Y_dec_train, channel_names=all_channel_names)

# Model

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization
from keras.layers import Convolution2D, MaxPooling3D, Flatten, Reshape
from keras.layers import TimeDistributed
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import InputLayer

tf.keras.backend.set_image_data_format('channels_last')

In [ ]:
# Run Experiment 2: No-fire December set + HRRR
print("\n==== Running Experiment 2: No-fire December set + HRRR ====")
print(f"Training data shape: {X_exp2_train_scaled.shape}")
print(f"Target data shape: {Y_dec_train.shape}")

# Build model
seq = Sequential()

seq.add(
    InputLayer(shape=X_exp2_train_scaled.shape[1:])
)

seq.add(
    ConvLSTM2D(
            filters=15, 
            kernel_size=(3, 3),
            padding='same', 
            return_sequences=True
    )
)

seq.add(
    ConvLSTM2D(
        filters=30, 
        kernel_size=(3, 3),
        padding='same', 
        return_sequences=True
    )
)

seq.add(
    Conv3D(
        filters=15, 
        kernel_size=(3, 3, 3),
        activation='relu',
        padding='same'    
    )
)

seq.add(
    Conv3D(
        filters=1, 
        kernel_size=(3, 3, 3),
        activation='relu',
        padding='same'
    )
)

seq.add(Flatten())
seq.add(Dense(Y_dec_train.shape[1], activation='relu'))

# Compile model
seq.compile(loss='mean_absolute_error', optimizer='adam')

# Print model summary
seq.summary()

# Train model
print(f"\nTraining model...")
epochs = 100  # Set to 100 to match Experiment 1
batch_size = 4
history = seq.fit(
    X_exp2_train_scaled, Y_dec_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

# Evaluate model
print(f"\nEvaluating model...")
test_loss = seq.evaluate(X_exp2_test_scaled, Y_dec_test, verbose=0)
print(f"Test MAE: {test_loss:.4f}")

# Make predictions
y_pred = seq.predict(X_exp2_test_scaled, verbose=0)

# Calculate metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(Y_dec_test, y_pred)
rmse = np.sqrt(mean_squared_error(Y_dec_test, y_pred))
r2 = r2_score(Y_dec_test, y_pred)

print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
plt.title('Experiment 2: No-fire December set + HRRR\nTraining Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MAE)')
plt.grid(True, alpha=0.3)
plt.show()

# Save results
results_dir = os.path.join(output_dir, "experiment2", "results")
os.makedirs(results_dir, exist_ok=True)

np.save(os.path.join(results_dir, "y_pred.npy"), y_pred)
seq.save(os.path.join(results_dir, "model.h5"))

exp2_results = {
    'model': seq,
    'history': history,
    'loss': test_loss,
    'mae': mae,
    'rmse': rmse,
    'r2': r2,
    'y_pred': y_pred,
    'y_test': Y_dec_test
}

# Evaluate

In [ ]:
print(f"\nDetailed analysis for Experiment 2:")
X_test = X_exp2_test_scaled
y_test = Y_dec_test
y_pred = exp2_results['y_pred']
model = exp2_results['model']
description = "No-fire December set + HRRR, two weeks"

print(f"Analyzing Experiment 2: {description}")

In [ ]:
from libs.plotting import (
    plot_prediction_comparison,
    plot_scatter_comparison,
    plot_error_by_sensor,
    plot_time_series_comparison,
    plot_input_frames,
    print_metrics
)

# Sensor names (use AirNow sensor names if available)
if hasattr(dec_airnow, 'sensor_names') and dec_airnow.sensor_names is not None:
    sensor_names = dec_airnow.sensor_names
else:
    sensor_names = [
        "North Holywood", 
        "Los Angeles - N. Main Street", 
        "Compton",
        "Crestline - Lake Gregory",
        "Fontana - Arrow Highway",
        "Glendora - Laurel",
        "Lake Elsinore - W. Flint Street",
        "Long Beach Signal Hill",
        "Mira Loma - Van Buren",
        "Reseda",
        "Riverside - Rubidoux",
        "Santa Clarita",
        "Simi Valley - Cochran Street",
        "Temecula (Lake Skinner)"
    ]

print("\n1. Plotting prediction comparison...")
plot_prediction_comparison(y_pred, y_test, sensor_names, sample_idx=8)

print("\n2. Plotting scatter comparison...")
plot_scatter_comparison(y_pred, y_test)

print("\n3. Plotting error by sensor...")
plot_error_by_sensor(y_pred, y_test, sensor_names)

print("\n4. Plotting time series comparison...")
plot_time_series_comparison(y_pred, y_test, sensor_names)
    
print("\n5. Plotting time series with shifted predictions...")
plot_time_series_comparison(y_pred, y_test, sensor_names, shift_pred=1)

print("\n6. Printing metrics...")
print_metrics(y_pred, y_test, sensor_names)

In [ ]:
# Save experiment comparison
with open(os.path.join(output_dir, 'experiment2_results.txt'), 'w') as f:
    f.write("==== Experiment 2 Results ====\n")
    f.write(f"Experiment 2 (No-fire December + HRRR, two weeks): MAE = {exp2_results['mae']:.4f}, RMSE = {exp2_results['rmse']:.4f}, R² = {exp2_results['r2']:.4f}\n")
    f.write(f"\nAnalysis completed on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\nExperiment 2 complete!")